In [1]:
!pip install dlib
!pip install imutils
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
!wget https://www.soundjay.com/buttons/sounds/button-3.wav -O alarm.wav
# Import necessary libraries
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, Audio
from google.colab.output import eval_js
from base64 import b64decode

# --- JavaScript and Python Functions for Webcam Access ---

def video_stream():
  """
  Starts the webcam stream and displays it in the Colab output.
  """
  js = Javascript('''
    var video;
    var div = null;
    var stream;

    // Starts the video stream and creates the necessary DOM elements
    async function startVideoStream() {
      // Create a container div
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      // Add a label
      const instruction = document.createElement('div');
      instruction.innerHTML = "Click on the video stream to stop.";
      div.appendChild(instruction);

      // Create the video element
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { stopVideoStream(); }; // Stop stream on click
      div.appendChild(video);

      // Get webcam access
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "user"}});
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Store variables on the window object to be globally accessible
      window.video = video;
      window.stream = stream;
      window.div = div;
    }

    // Stops the video stream and removes the DOM elements
    function stopVideoStream() {
      if (window.stream) {
        window.stream.getTracks().forEach(track => track.stop());
      }
      if (window.div) {
        window.div.remove();
      }
      window.video = null;
      window.stream = null;
      window.div = null;
    }

    // Start the stream automatically
    startVideoStream();
  ''')
  display(js)

def video_frame():
  """
  Captures a single frame from the webcam stream.
  """
  js_code = '''
    async function captureFrame() {
      // Return null if the video stream is not active
      if (!window.video) {
        return null;
      }
      // Create a canvas to draw the frame on
      const canvas = document.createElement('canvas');
      canvas.width = window.video.videoWidth;
      canvas.height = window.video.videoHeight;
      const context = canvas.getContext('2d');
      context.drawImage(window.video, 0, 0, canvas.width, canvas.height);

      // Return the frame as a data URL
      return canvas.toDataURL('image/jpeg', 0.8);
    }
    captureFrame();
  '''
  try:
    data_url = eval_js(js_code)
    if data_url is None:
      return None

    # Decode the base64 image data
    header, encoded = data_url.split(',', 1)
    img_bytes = b64decode(encoded)
    np_arr = np.frombuffer(img_bytes, dtype=np.uint8)
    frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return frame
  except Exception as e:
    # This can happen if the user stops the stream
    return None

def video_release():
    """Releases the webcam and cleans up the output."""
    eval_js('stopVideoStream()')

def eye_aspect_ratio(eye):
    """Computes the eye aspect ratio (EAR)."""
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

--2025-11-21 10:24:45--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-11-21 10:24:46--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  28.7MB/s    in 2.1s    

2025-11-21 10:24:48 (28.7 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

--2025-11-21 10:24:57--  https://www.soundjay.com/buttons/sounds/button-3.wav
Resolving www.soundjay.com (www.soundjay.com)... 192.163.208.117
Connecting to www.soundjay.com (www.s